# Radio Galaxy DINO

### 1. Install All the packages
This Note Book Utilizes Fast AI and DINO to train a ResNet 50 Network for the segmentation and object detectiontasks.

In [1]:
# Download the packages.
#!pip install galaxy-datasets[pytorch]
!pip install fastai
!pip install self-supervised

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [2]:
from torchvision.transforms import v2 as T
import matplotlib.pyplot as plt
import torch
#from galaxy_datasets.pytorch import GZ2, GZHubble, GZDecals5
from fastai.vision.all import *
from self_supervised.augmentations import *
from self_supervised.layers import *
from self_supervised.models.vision_transformer import *
from self_supervised.vision.dino import *
from self_supervised.vision.swav import get_swav_aug_pipelines
from torchvision.transforms import ToTensor

import torch
import numpy as np
import torch.nn as nn
from torchvision.models.resnet  import resnet50
from torch.utils.data import ConcatDataset
from torch.utils.data import Dataset, DataLoader, Sampler
from torchvision import datasets, transforms

## 2. Mount Drive

In [3]:
from google.colab import drive

drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [4]:
!cp 'gdrive/MyDrive/FYP/radio_data.zip' 'radio_data.zip'

In [5]:
!unzip "radio_data.zip" -d ""

Streaming output truncated to the last 5000 lines.
  inflating: mirabest_aug2/train/7263.png  
  inflating: mirabest_aug2/train/7264.png  
  inflating: mirabest_aug2/train/7265.png  
  inflating: mirabest_aug2/train/7266.png  
  inflating: mirabest_aug2/train/7267.png  
  inflating: mirabest_aug2/train/7268.png  
  inflating: mirabest_aug2/train/7269.png  
  inflating: mirabest_aug2/train/727.png  
  inflating: mirabest_aug2/train/7270.png  
  inflating: mirabest_aug2/train/7271.png  
  inflating: mirabest_aug2/train/7272.png  
  inflating: mirabest_aug2/train/7273.png  
  inflating: mirabest_aug2/train/7274.png  
  inflating: mirabest_aug2/train/7275.png  
  inflating: mirabest_aug2/train/7276.png  
  inflating: mirabest_aug2/train/7277.png  
  inflating: mirabest_aug2/train/7278.png  
  inflating: mirabest_aug2/train/7279.png  
  inflating: mirabest_aug2/train/728.png  
  inflating: mirabest_aug2/train/7280.png  
  inflating: mirabest_aug2/train/7281.png  
  inflating: mirabest_aug2/

## 3. Define Dataset and Dataloaders

In [6]:
# Define the dataset used, apply the same transformation on the dataset.

class UnlabeledDataset(Dataset):
    def __init__(self, directory, transform=None):
        """
        Args:
            directory (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.directory = directory
        self.transform = transform
        self.image_files = [os.path.join(directory, file) for file in os.listdir(directory)]

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        image_path = self.image_files[idx]
        image = Image.open(image_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        return image

class CustomSampler(Sampler):
    def __init__(self, data_source1, data_source2, p=0.7):
        # This stores the indices of both datasets concatenated
        self.indices1 = np.arange(len(data_source1))
        self.indices2 = np.arange(len(data_source2)) + len(data_source1)  # Offset by the size of the first dataset
        self.p = p  # Probability of choosing from dataset1
        self.total_size = len(data_source1) + len(data_source2)

    def __iter__(self):
        # Shuffle indices
        np.random.shuffle(self.indices1)
        np.random.shuffle(self.indices2)

        # Decide how many samples to draw from each dataset
        count1 = int(self.total_size * self.p)
        count2 = self.total_size - count1

        # Yield indices from each dataset according to the calculated counts
        yield from self.indices1[:count1]
        yield from self.indices2[:count2]

    def __len__(self):
        return self.total_size

In [7]:
# Define dataset Transformation and dataloader collate.
transformations = transforms.Compose([
    transforms.Resize((450, 450)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_dataset_mb = UnlabeledDataset('mirabest_aug2/train', transform=transformations)
train_dataset_rgn = UnlabeledDataset('mirabest_aug2/train_dino', transform=transformations)
train_dataset = ConcatDataset([train_dataset_mb, train_dataset_rgn])
custom_sampler = CustomSampler(train_dataset_rgn, train_dataset_mb, p=0)

#custom_sampler = CustomSampler(train_dataset_rgn, train_dataset_mb, p=1)
val_dataset = UnlabeledDataset('mirabest_aug2/val_dino', transform=transformations)
# Create data loader
train_data_loader =  torch.utils.data.DataLoader(train_dataset, batch_size=64, num_workers=16, sampler=custom_sampler)
test_data_loader =  torch.utils.data.DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=16)
data = DataLoaders(train_data_loader, test_data_loader)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


## 4. Create the ResNet Model To be Trained

In [8]:
import torchvision.models.detection as detection_models

def create_resnet50_coco_backbone(isteacher = True):
  if isteacher == True:
    faster_rcnn_model = detection_models.fasterrcnn_resnet50_fpn(pretrained=True)
    extracted_backbone = faster_rcnn_model.backbone.body
    backbone = resnet50(pretrained=False)
    backbone.fc = nn.Identity()
    backbone.load_state_dict(extracted_backbone.state_dict(), strict=True)

  else:
    backbone = resnet50(pretrained=False)
    backbone.fc = nn.Identity()
  return backbone


# Initialize backbones for both student and teacher
resnet50_backbone_student = create_resnet50_coco_backbone(True)
resnet50_backbone_student = MultiCropWrapper(resnet50_backbone_student)
resnet50_backbone_teacher = create_resnet50_coco_backbone(True)
resnet50_backbone_teacher = MultiCropWrapper(resnet50_backbone_teacher)

# Initialize the heads for student and teacher
dino_head = DINOHead(2048, 2**16, norm_last_layer=True)

# Create the full models
student_model = nn.Sequential(resnet50_backbone_student, dino_head)
teacher_model = nn.Sequential(resnet50_backbone_teacher, dino_head)

# Assuming DINOModel is defined elsewhere and handles the logic specific to DINO
dino_model = DINOModel(student_model, teacher_model)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth
100%|██████████| 160M/160M [00:00<00:00, 224MB/s]
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None

## 6. Training Loop

### 6.1 Data-augmentation

In [9]:
aug_pipelines = get_dino_aug_pipelines(rotate=True,
                                       bw=True,
                                       crop_sizes=(450, 180), # Define the maximum crop threshold,
                                       min_scales=(0.4, 0.05),
                                       max_scales=(1.0, 0.4),
                                       rotate_deg=360,
                                       jitter=True, # Color Jitter
                                       jitter_s=0.5,
                                       jitter_p=0.8,
                                       blur=True, # Blur
                                       blur_p=0.1,
                                       blur_s=(4, 16))

### 6.2 Save Model Class

In [10]:
class SaveModelCallback(TrackerCallback):
    "A `TrackerCallback` that saves the model's best during training and loads it at the end."
    _only_train_loop,order = True,TrackerCallback.order+1
    def __init__(self, monitor='valid_loss', comp=None, min_delta=0., fname='model', every_epoch=False, at_end=False,
                 with_opt=False, reset_on_fit=True):
        super().__init__(monitor=monitor, comp=comp, min_delta=min_delta, reset_on_fit=reset_on_fit)
        assert not (every_epoch and at_end), "every_epoch and at_end cannot both be set to True"
        # keep track of file path for loggers
        self.last_saved_path = None
        store_attr('fname,every_epoch,at_end,with_opt')
        #!cp -r '/models' '/gdrive/MyDrive/models_dino'

    def _save(self, name): self.last_saved_path = self.learn.save(name, with_opt=self.with_opt)

    def after_epoch(self):
        "Compare the value monitored to its best score and save if best."
        if self.every_epoch:
            if (self.epoch%self.every_epoch) == 0: self._save(f'{self.fname}_{self.epoch}')
        else: #every improvement
            super().after_epoch()
            if self.new_best:
                print(f'Better model found at epoch {self.epoch} with {self.monitor} value: {self.best}.')
                self._save(f'{self.fname}')

    def after_fit(self, **kwargs):
        "Load the best model."
        if self.at_end: self._save(f'{self.fname}')
        elif not self.every_epoch: self.learn.load(f'{self.fname}', with_opt=self.with_opt)

In [ ]:
#torch.cuda.empty_cache()

### 6.3 Training Loop

In [12]:
# Define the DINO pipeline
#tpt: student team temperature
max_epochs = 12

dino_cb = DINO(aug_pipelines=aug_pipelines,
               tpt_start=0.03,
               tpt_end=0.045,
               tpt_warmup_pct=0.,
               freeze_last_layer=max_epochs)

grad_clip_cb = GradientClip(max_norm=3., norm_type=2.)
save_cb = SaveModelCallback(every_epoch=1, with_opt=True, fname='dino_radio_pretraining_ft')
nan_cb = TerminateOnNaNCallback()

cbs=[dino_cb, grad_clip_cb, save_cb, nan_cb]

# Set up the Learner Module Used to train the Model
learn = Learner(data, dino_model, loss_func=F.nll_loss, opt_func=Adam, cbs=cbs)
learn.to_fp16();


max_lr = 2.5e-5
# 2.5e-5
# 1e-4
lr_sched = combine_scheds([0.2,0.8], [SchedLin(0.,max_lr), SchedCos(max_lr,1e-6)])
wd_sched = SchedCos(0.04,0.4)
param_scheduler = ParamScheduler({"lr":lr_sched, "wd":wd_sched})
learn.fit(max_epochs, cbs=[param_scheduler])


epoch,train_loss,valid_loss,time
0,11.146365,10.605776,03:58
1,10.843534,10.193938,03:57
2,10.540680,4.245701,03:58
3,10.054310,1.620525,03:58
4,9.919879,1.587745,03:58
5,9.506135,1.576017,03:58
6,8.811870,1.576646,03:58
7,7.920749,1.611914,03:58
8,7.319016,1.612623,03:58
9,6.973163,1.601682,03:58
